In [2]:
import os

try:
    from dotenv import load_dotenv
except ImportError:
    from pip._internal import main as pip
    pip(['install', 'python-dotenv'])
    from dotenv import load_dotenv
try:
    import psycopg2
except ImportError:
    from pip._internal import main as pip
    pip(['install', 'psycopg2'])
    import psycopg2
    
import sqlalchemy
import pandas as pd

load_dotenv(
    dotenv_path='./.env')
db = os.getenv('DB')

sql = sqlalchemy.text("""SELECT CASE
           WHEN sub_event_type = 'Air/drone strike' THEN 'Drone Strike'
       END type_of_strike,
       acled.*,
       ST_Distance(acled.geom::geography,cities.geom::geography) AS distance_to_nearest_city,
       cities.name AS nearest_city,
       cities.gn_pop
FROM acled
CROSS JOIN LATERAL
    (SELECT name,
            geom,
            cities.gn_pop
     FROM cities
     WHERE gn_pop > 0
     ORDER BY cities.geom <-> acled.geom
     LIMIT 1) cities
WHERE country IN ('Afghanistan',
                  'Syria',
                  'Iraq',
                  'Yemen',
                  'Pakistan',
                  'Mali',
                  'Turkey',
                  'Ukraine',
                  'Azerbaijan',
                  'Saudi Arabia',
                  'Palestine',
                  'Armenia',
                  'Libya',
                  'Burkina Faso',
                  'Somalia',
                  'Egypt',
                  'Israel',
                  'Lebanon',
                  'Venezuela',
                  'United Arab Emirates',
                  'Nigeria',
                  'South Sudan')
    AND year > 2000
    AND year < 2021
    AND sub_event_type = 'Air/drone strike'
    AND notes LIKE '%drone%'
UNION
SELECT CASE
           WHEN sub_event_type = 'Air/drone strike' THEN 'Air Strike'
       END type_of_strike,
       acled.*,
       ST_Distance(acled.geom::geography,cities.geom::geography) AS distance_to_nearest_city,
       cities.name AS nearest_city,
       cities.gn_pop
FROM acled
CROSS JOIN LATERAL
    (SELECT name,
            geom,
            cities.gn_pop
     FROM cities
     WHERE gn_pop > 0
     ORDER BY cities.geom <-> acled.geom
     LIMIT 1) cities
WHERE country IN ('Afghanistan',
                  'Syria',
                  'Iraq',
                  'Yemen',
                  'Pakistan',
                  'Mali',
                  'Turkey',
                  'Ukraine',
                  'Azerbaijan',
                  'Saudi Arabia',
                  'Palestine',
                  'Armenia',
                  'Libya',
                  'Burkina Faso',
                  'Somalia',
                  'Egypt',
                  'Israel',
                  'Lebanon',
                  'Venezuela',
                  'United Arab Emirates',
                  'Nigeria',
                  'South Sudan')
    AND year > 2000
    AND year < 2021
    AND sub_event_type = 'Air/drone strike'
    AND notes NOT ILIKE ALL (ARRAY['%drone%'])
ORDER BY event_date;""")

df = pd.read_sql_query(sql, db)


In [ ]:
df['country'].unique()

In [ ]:
df[df['seller'] == ('China'or'Iran'or'Turkey'or
                    'United Arab Emirates'or'United States')].head()


In [ ]:
seller_list = ['China', 'Iran', 'Turkey',
               'United Arab Emirates', 'United States']


In [ ]:
df[['delivery_start_year','delivery_end_year']] = df['delivery_year'].str.split('-',expand=True)


In [ ]:
df['delivery_end_year'].fillna(df['delivery_start_year'], inplace=True)


In [ ]:
df['delivery_end_year'] = df.apply(
    lambda x: pd.to_datetime(x['delivery_end_year'],format='%Y'), axis=1)


In [ ]:

df['delivery_start_year'] = df.apply(
    lambda x: pd.to_datetime(x['delivery_start_year'], format='%Y'), axis=1)


In [ ]:
print(abs(df['delivery_end_year'][0] - df['delivery_start_year'][0]).total_seconds())


In [ ]:
df['delivery_end_year']


In [ ]:
df['seller'].unique()

In [ ]:
df['buyer'].unique()


In [ ]:
len(df)

In [ ]:
len(df['buyer'].unique())


In [ ]:
df.query("weapon_description == 'Armed UAV'")


In [3]:
import plotly.express as px


In [14]:
#df.groupby(['year','type_of_strike'])['type_of_strike'].count()

newdf = df.groupby(['year','type_of_strike'])['type_of_strike'].count().reset_index(name='count').sort_values(['year'], ascending=True)

In [34]:
fig = px.bar(newdf, y='count', x='year',color='type_of_strike',
title='ACLED Drone Strikes and Air Strikes from 2000 to 2021', 
labels={'count':'Counts', 'year':'Year','type_of_strike':'Type of Strikes'})
fig.update_traces(
    texttemplate='%{y:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=6,uniformtext_mode='hide')
fig.update_xaxes(dtick='Y1')
fig.update_layout(barmode='group')
fig.show()


In [35]:
air_means = []
drone_means = []

for __ in range(1000):
    boot_sample = df.query('type_of_strike == "Drone Strike"').sample(
        len(df.query('type_of_strike == "Drone Strike"')), replace=True)
    drone_means.append(boot_sample.fatalities.mean())

for __ in range(1000):
    boot_sample = df.query('type_of_strike == "Air Strike"').sample(
        len(df.query('type_of_strike == "Air Strike"')), replace=True)
    air_means.append(boot_sample.fatalities.mean())


In [38]:
bootstrap = pd.DataFrame(list(zip(air_means,drone_means)),
columns=['Air Strikes','Drone Strikes'])

In [42]:
fig = px.histogram(bootstrap, x=['Air Strikes', 'Drone Strikes'],
                   marginal="box",labels={'value':'Average Fatality',
                   'variable':'Type of Strike'},
                   title='Bootstrapped Fatalities Between <br> Air Strikes and Drone Strikes from 2000 to 2021')
fig.show()


In [43]:
countrydf = df.groupby(['country', 'type_of_strike'])['type_of_strike'].count(
).reset_index(name='count').sort_values(['count'], ascending=True)


In [45]:
fig = px.bar(countrydf, y='count', x='country', color='type_of_strike',
             title='ACLED Drone Strikes and Air Strikes from 2000 to 2021 by Country',
             labels={'count': 'Counts', 'country': 'Country', 'type_of_strike': 'Type of Strikes'},
             category_orders={'type_of_strike':['Air Strike', 'Drone Strike']})
fig.update_traces(
    texttemplate='%{y:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=6, uniformtext_mode='hide')
fig.update_layout(barmode='group')
fig.show()


In [55]:
df_drone = df.query('type_of_strike == "Drone Strike"')

fig = px.scatter_mapbox(df_drone, lat="latitude", lon='longitude', color='type_of_strike',
                        size="fatalities", zoom=2, title='ACLED Drone Strikes from 2000 to 2021', labels={'type_of_strike': 'Type of Strike'},
                        color_discrete_sequence=['red'])
fig.update_layout(mapbox_style="open-street-map")
fig.show()
